Функции бывают недетерминированными, т.е. они могут возвращать разные ответы для одного и того же набора аргументов.

In [130]:
import numpy as np

In [131]:
np.random.randint(1, 10)

4

In [132]:
np.random.randint(1, 10)

9

Но хаос можно контролировать, указав seed

In [133]:
np.random.seed(42)

np.random.randint(1, 10) # у меня вернуло 7

7

In [134]:
np.random.seed(42)

np.random.randint(1, 10)  # у меня опять вернуло 7

7

In [135]:
np.random.seed(42)

np.random.randint(1, 10)  # и снова вернуло 7!

7

Замечание: внимание, тут всё немного хитрее. Фиксированный сид не значит, что np.random.randint(1, 10) всегда выводит одно и то же!

In [136]:
np.random.seed(42)

for _ in range(3):
    print(np.random.randint(1, 10))  # 7 4 8


7
4
8


Фиксированный сид не значит, что последовательность значений np.random.randint(1, 10) всегда одинаковая.

In [137]:
np.random.seed(42)

for _ in range(3):
    print(np.random.randint(1, 10))  # опять 7 4 8


7
4
8


Является ли LinearRegression детерминированной? Как мы неожиданно выяснили, LinearRegression вовсе не использует градиентный бустинг из курсеры, а решает задачу методами линейной алгебры. Вполне возможно, что она детерминирована. Но давайте проверим. 

In [138]:
# качаем наш датасет

import kagglehub

dataset_local_folder = kagglehub.dataset_download("jakewright/house-price-data")
dataset_local_csv = f"{dataset_local_folder}/kaggle_london_house_price_data.csv"

import pandas as pd

df = pd.read_csv(dataset_local_csv)
df.dropna(subset=['floorAreaSqM', 'saleEstimate_currentPrice'], inplace=True)

In [139]:
from sklearn.linear_model import LinearRegression

X = df[['floorAreaSqM']]
y = df['saleEstimate_currentPrice']
    

In [140]:
model = LinearRegression()
model.fit(X, y)
print(model.coef_[0], model.intercept_)

11364.96911227819 -251334.1616502545


In [141]:
model = LinearRegression()
model.fit(X, y)
print(model.coef_[0], model.intercept_)

11364.96911227819 -251334.1616502545


Вроде одно и то же! Ура! Давайте убедимся, что бывает и по-разному!

In [142]:
# это другой алгоритм, но тоже линейный, зато наверняка недетерминированный
from sklearn.linear_model import SGDRegressor

In [143]:
model = SGDRegressor()
model.fit(X, y)
print(model.coef_[0], model.intercept_[0])

67823766.57967646 1761061509.3041806


In [144]:
model = SGDRegressor()
model.fit(X, y)
print(model.coef_[0], model.intercept_[0])

-91874179.35728526 91649144.34769498


Вроде теперь получается разное! Но и тут можно зафиксировать seed!

In [145]:
model = SGDRegressor(random_state=42)
model.fit(X, y)
print(model.coef_[0], model.intercept_[0])

161006.19608022354 639026.7184629464


In [146]:
model = SGDRegressor(random_state=42)
model.fit(X, y)
print(model.coef_[0], model.intercept_[0])

161006.19608022354 639026.7184629464


Теперь одно и то же! Ещё мы говорили о тестировании.

In [147]:
from sklearn.model_selection import train_test_split

# разобьем наш датасет на обучающую и тестовую выборки
X_learn, X_test, y_learn, y_test = train_test_split(X, y, test_size=0.1)

# вот так можно посчитать ошибку модели руками
# конечно если и готовая функция, и именно её и нужно использовать, но так не очень наглядно
def evaluate_model(model, X_test, y_test):
    error = 0
    for i in range(len(X_test)):
        # тут всё приходится немного выкручиваться, потому что все эти библиотеки очень не любят циклы по отдельным строчкам
        X_single = X_test.iloc[[i]] 
        y_pred = model.predict(X_single)[0]
        error += (y_pred - y_test.iloc[i]) ** 2
    error /= len(X_test)  # Среднеквадратичная ошибка
    return error

# правильнее конечно, вот так
"""
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)  # Предсказания для всей выборки
    error = np.mean((predictions - y_test) ** 2)  # Среднеквадратичная ошибка
    return error
"""

model1 = LinearRegression()
model1.fit(X_learn, y_learn)
error1 = evaluate_model(model1, X_test, y_test)

model2 = SGDRegressor(random_state=42)
model2.fit(X_learn, y_learn)
error2 = evaluate_model(model2, X_test, y_test)

print(error1, error2)
if error1 < error2:
    print("Model 1 is better")
else:
    print("Model 2 is better")

307964208100.029 8.249607145113637e+23
Model 1 is better


ДЗ. Запустите код выше несколько раз и ответьте на следующие вопросы:

1) Почему ошибки при каждом запуске разные? Ведь обе модели LinearRegression() и SGDRegressor(random_state=42) детерминированные!
2) Как это исправить?
3) Почему первая ошибка стабильно меньше второй?